# Installing neccessary packages

In [1]:
library(tidyverse)
library(lubridate)

library(jsonlite)
library(httr)
library(rvest)

In [2]:
trimStringArray = function(array) {
  if (is.array(array)) {
    for (i in 1:length(array())) {
      array[i] = trimws(array[i])
    }
  } else {
    array = trimws(array)
  }
  array
}

In [3]:
scrapeColumnsWM = function(game_name) {

  # base url

  url = 'https://www.transfermarkt.com/weltmeisterschaft-2022/teilnehmer/pokalwettbewerb/WM22/saison_id/2021'

  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header
  slugs = c()
  club_ids = c()

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    item %>%
      html_elements('.table-header') %>%
      html_text() %>%
      trimStringArray() -> name

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # scrape the url with the slugs and club ids of the countries

      responsive_table %>%
          html_elements('tr td:first-of-type a') %>%
          html_attr('href') %>%
          strsplit('/') -> raw_url_data

      raw_url_data

      # obtain the slug of all countries

      raw_url_data[1][[1]][2]
      raw_url_data[1][[1]][length(raw_url_data[1][[1]])]

      for (i in 1:length(raw_url_data)) {
          slugs = c(slugs, raw_url_data[i][[1]][2])
          club_ids = c(club_ids, raw_url_data[i][[1]][length(raw_url_data[1][[1]])])
      }

      # print slugs

      slugs

      # print club_ids

      club_ids

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames

      # add slug and club_id to columnNames

      columnNames = c(columnNames, c('slug', 'club_id'))

      # print columnNames again

      columnNames

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              print(paste0("Column ", i, " is not a valid value"))
          }

      }

      # add club_id and slug to values

      columnValues = append(columnValues, list(slugs))
      columnValues = append(columnValues, list(club_ids))

      # print columnValues

      columnValues

      # return list
      list(columnNames, columnValues)
  } else {
      # return NULL
      NULL
  }
}

In [4]:
create_dataframeWM = function(name) {
  print(scrapeColumnsWM(name))
  df_wm = scrapeColumnsWM(name)

  if(!(is.null(df_wm))) {
      columns = df_wm[[2]]
      columnNames = df_wm[[1]]

      #print(columnNames)

      df_wm = data.frame(columns[[1]])

      for (i in 2:length(columns)) {
          df_wm[i] = columns[[i]]
      }

      names(df_wm) = columnNames[2:length(columnNames)]
      df_wm
  } else {
      NULL
  }
}

In [5]:
df_wm = create_dataframeWM('teilnehmende teams an der wm')
df_wm

In [8]:
df_wm$Club -> teilnehmer

teilnehmer

# Import csv file "International football results from 1872 to 2022" 

In [9]:
results <- read.csv2("https://raw.githubusercontent.com/martj42/international_results/master/results.csv", header = TRUE, sep = ",")
results

# Select results since 2010

In [10]:
results %>%
    filter(date >=2010) %>%
    select(-country, -city) -> results
results

# Select results from 32 participants 

In [11]:
myFunc = function(country) {
    results %>%
      filter(home_team == country) -> result_home

    results %>%
      filter(away_team == country) -> result_away

    bind_rows(result_home, result_away) -> result

    result %>%
      arrange(date) %>%
      mutate(country = country) -> result
    
    # return
    result
}

In [12]:
myFunc('Wales')

# Combine all dataframes into one

In [13]:
new_football_df = map_df(teilnehmer, myFunc)

new_football_df

In [12]:
# combine all dataframes into one dataframe, delete the repeated rows
results_selected <- bind_rows(result_qatar, result_ecuador, result_senegal, result_netherlands, result_england, result_iran, result_unitedstates, result_wales, result_argentina, result_saudiarabia, result_mexico, result_poland, result_france, result_australia, result_denmark, result_spain, result_costarica, result_germany, result_japan, result_belgium, result_canada, result_morocco, result_croatia, result_brazil, result_serbia, result_switzerland, result_cameroon, result_portugal, result_ghana, result_uruguay, result_southkorea)

results_selected %>%
  arrange(date) -> results_selected

results_selected

# Continue process dataframe

In [ ]:
#compare scores from home and away teams
  results_selected %>%
    mutate(Status = home_score - away_score) -> result_processed_1

#get finalresults of the games, focused on home teams
  mutate(result_processed_1, 
         HomeFinalStatus = ifelse(Status > 0, "Win", ifelse(Status < 0, "Lose", "Draw"))) -> result_processed_2
  
  mutate(result_processed_2, 
         AwayFinalStatus = ifelse(Status > 0, "Lose", ifelse(Status < 0, "Win", "Draw"))) -> result_processed_2

#confirm if home team is country or not
  mutate(result_processed_2, 
         HomeCountry = ifelse(country == home_team, "TRUE", "FALSE")) -> result_processed_3

#confirm FinalResults accroding to finalstatus and homecountry
  mutate(result_processed_3, 
         FinalResult = ifelse(HomeCountry == "TRUE", HomeFinalStatus, AwayFinalStatus)) -> result_processed_4

#reduce redundant colomns
  result_processed_4 %>%
    select(-Status, -HomeFinalStatus, -AwayFinalStatus, -HomeCountry) -> result_processed_5

result_processed_5